<a href="https://colab.research.google.com/github/olonok69/projects_ai/blob/main/Prompt_OLLama_DeepSeek_Phi3_5_LLama3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture output
! pip install -r /content/drive/MyDrive/models/requirements.txt -qqq

In [3]:

%%capture output
! pip install langchain-ollama "ollama==0.4.2" -qqq

# Install Ollama
- curl https://ollama.ai/install.sh | sh

- Run server with: ollama serve &

- Pull model ollama: pull llama3:8b

In [24]:
import pprint
import pandas as pd
import json

In [28]:
session_data_path = "/content/drive/MyDrive/data/CSM/speaker_aggregated_info.json" # For Recommendation engine
session_data_path = "/content/drive/MyDrive/data/CSM/badge_id_aggregated_results.json"
nomenclature_embeddings_path = "/content/drive/MyDrive/data/CSM/cluster_numeculature.json"
output_path = "/content/drive/MyDrive/data/CSM/badge/embeddings_badges_mistral.json"

csv_path = '/content/drive/MyDrive/data/CSM/new_claire_db_badge_cluster_data_with_aggregated_info_GIO_GT_LABELS.csv'
#csv_path = '/content/drive/MyDrive/data/CSM/20240512_new_labels_WITH_AGGINFO.csv'

In [29]:
data = pd.read_csv(csv_path)
data.head()

,BadgeId,ClusterId,AggregatedInfo
0,VPWNB8E,Sourcing – Early,Show Ref: BDAIWL24. Badge Id: VPWNB8E. Registr...
1,54ZPEZG,Learning,Show Ref: CEEL24. Badge Id: 54ZPEZG. Registrat...
2,GGG4EI4,Searching,Show Ref: BDAIWL24. Badge Id: GGG4EI4. Registr...
3,X3PDT43,Learning,Show Ref: CEEL24. Badge Id: X3PDT43. Registrat...
4,49CCFCJ,Searching,Show Ref: BDAIWL24. Badge Id: 49CCFCJ. Registr...


In [13]:
len(data)

1215

In [30]:
data.ClusterId.value_counts()

,count
ClusterId,
Networking,243
Learning,231
Sourcing – Early,208
Searching,204
Sourcing – In Process,100
Sourcing – Deciding,14


In [31]:
df_networking = data[data['ClusterId'] == 'Networking']
df_learning = data[data['ClusterId'] == 'Learning']
df_searching = data[data['ClusterId'] == 'Searching']
df_sourcing_early = data[data['ClusterId'] == 'Sourcing – Early']
df_sourcing_inprocess = data[data['ClusterId'] == 'Sourcing – In Process']
df_sourcing_deciding = data[data['ClusterId'] == 'Sourcing – Deciding']

In [32]:
example_networking = df_networking[0:1].to_dict()
example_learning = df_learning[0:1].to_dict()
example_searching = df_searching[0:1].to_dict()
example_sourcing_early = df_sourcing_early[0:1].to_dict()
example_sourcing_inprocess = df_sourcing_inprocess[0:1].to_dict()
example_sourcing_deciding = df_sourcing_deciding[0:1].to_dict()


In [54]:
list_examples = [example_networking, example_learning, example_searching, example_sourcing_early, example_sourcing_inprocess, example_sourcing_deciding]

In [60]:
examples={}
for example in list_examples:

  examples[list(example['ClusterId'].values())[0]] = list(example['AggregatedInfo'].values())[0]

In [61]:
examples.keys()

dict_keys(['Networking', 'Learning', 'Searching', 'Sourcing – Early', 'Sourcing – In Process', 'Sourcing – Deciding'])

In [33]:
df_networking=df_networking[1:]
df_learning=df_learning[1:]
df_searching=df_searching[1:]
df_sourcing_early=df_sourcing_early[1:]
df_sourcing_inprocess=df_sourcing_inprocess[1:]
df_sourcing_deciding=df_sourcing_deciding[1:]

In [25]:
with open(nomenclature_embeddings_path, 'r') as f:
    nomenclatures = json.load(f)

In [62]:
#nomenclatures

In [102]:
text_example = df_searching[0:1]['AggregatedInfo'].values[0]



In [103]:
# Function to generate the classification prompt
def generate_prompt(label_dict, examples, text_example):
    prompt = "Categories and Descriptions:\n\n"
    for label, description in label_dict.items():
        prompt += f"Category: {label} --> {description[len(label)+1:]}\n"

    prompt += "\nExamples: (1 text associated to a category)\n"
    for label, example in examples.items():
        prompt += f"Category: {label} --> {example}\n"

    prompt += "\nText to classify:\n"
    prompt += "----------------------------------------------------\n"
    prompt += text_example
    prompt += "----------------------------------------------------\n"
    prompt += "\n\n Only return a class label among these 6:\
        'Networking', 'Learning', 'Searching', 'Sourcing – Early', 'Sourcing – In Process', 'Sourcing – Deciding' "
    return prompt

In [104]:
prompt = generate_prompt(nomenclatures, examples, text_example)
len(prompt)

76262

In [105]:
print(prompt)

Categories and Descriptions:

Category: Networking -->  Visitors in this group are primarily focused on building professional relationships and expanding their business network. They attend events to meet industry peers, experts, and potential partners. Their interactions are more about establishing connections than immediate purchasing. They might be decision-makers or influencers in their organizations who value personal interactions and opportunities for collaboration.
Category: Learning -->  These attendees are motivated by educational opportunities. They seek to gain new insights, learn about industry trends, and enhance their knowledge and skills. They are likely to attend workshops, seminars, and keynote speeches. Their interest in products or services is secondary to their desire for professional development. They may represent a long-term opportunity as their increased knowledge could lead to future purchasing decisions.
Category: Searching --> (Gathering Information on Produc

# DeepSeek R1 Distill Qwen 1.5B

https://huggingface.co/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B

https://ollama.com/library/deepseek-r1

ollama run deepseek-r1:1.5b

Q4_K - Medium


In [70]:
from langchain_ollama import ChatOllama
from langchain_core.messages import AIMessage



In [106]:

llm = ChatOllama(
    model="deepseek-r1:1.5b",
    temperature=0.5,
    num_ctx=16000,

)
messages = [
    (
        "system",
        "You are a clever chatbot specialized in classify text.Only provide Label, no other Information. Let's think step by Step",
    ),
    ("human", prompt),
  ]
ai_msg = llm.invoke(messages)

print(ai_msg.content)
print("-"*100)

<think>
Alright, let's break this down. The user provided a bunch of data about a company and an event, along with their attendees. They want to know which class label from the list they provided (Networking, Learning, Searching, Sourcing – Early, Sourcing – In Process, Sourcing – Deciding) best fits this event.

First, I need to identify what kind of event this is. The event title is "Help us organize a virtual conference for Cloud Solutions." Looking at the data, it's an event related to cloud solutions and virtual events.

Now, looking at the attendees' details, most are in tech roles like IT, DevOps, Data Science, etc., which suggests it's about technology or software. The company is a tech company focused on Cloud Industry Solutions.

The event type seems to be a virtual conference for developers and tech professionals. In such events, the target audience is likely people interested in learning about the latest technologies, tools, and practices specific to their industry.

Consid

In [107]:
llm = ChatOllama(
    model="llama3.2:3b-instruct-q4_K_M",
    temperature=0.5,
    num_ctx=16000,

)
messages = [
    (
        "system",
        "You are a clever chatbot specialized in classify text.Only provide Label, no other Information. Let's think step by Step",
    ),
    ("human", prompt),
  ]
ai_msg = llm.invoke(messages)

print(ai_msg.content)
print("-"*100)

Based on the text, I would classify it as:

'Sourcing – Early'

Reasoning:
The text has a unique format and structure that is different from the other categories. It contains information about an attendee's company, job title, industry sector, and decision-making power, which suggests that it is related to sourcing or recruitment efforts. The fact that it is labeled as "Sourcing – Early" also implies that it is an early stage in the sourcing process, possibly indicating that the event is still in its planning stages or that the attendee has just registered.

Note: The other categories do not have enough information to make a confident classification, and 'Networking' is unlikely given the focus on sourcing and recruitment.
----------------------------------------------------------------------------------------------------


In [108]:
llm = ChatOllama(
    model="phi3.5",
    temperature=0.5,
    num_ctx=16000,

)
messages = [
    (
        "system",
        "You are a clever chatbot specialized in classify text.Only provide Label, no other Information. Let's think step by Step",
    ),
    ("human", prompt),
  ]
ai_msg = llm.invoke(messages)

print(ai_msg.content)
print("-"*100)

Show Ref: BDAIWL24. Badge Id: 49CCFCJ. Registration Date and other details provided do not give specific information about the attendee’s industry sector, job function, or interests related to sourcing events but mention their attendance as a representative/employee with no company-specific insights given either (implying networking), lack of detail on learning activities or searching for opportunities. The closest relevant class label from the provided options is 'Networking'. However, due to missing specific details about event engagement related to sourcing events and without clear indicators pointing towards a focus on acquiring knowledge ('Learning') or early stage exploration/search processes ('Searching'), it's challenging to definitively classify beyond the context of networking. Therefore:

Class Label: 'Networking'
----------------------------------------------------------------------------------------------------
